In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('missing').getOrCreate()
df = spark.read.csv('cleandiabetedata.csv', header=True, inferSchema=True)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/10 12:23:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["HighBP", "HighChol", "CholCheck", "BMI", "Smoker", "Stroke", "HeartDiseaseorAttack", "PhysActivity", "Fruits"\
              , "Veggies", "HvyAlcoholConsump", "GenHlth", "MentHlth", "PhysHlth", "DiffWalk", "Sex", "Age", "Education", "Income"],
    outputCol="features")

output = assembler.transform(df)
output.select("features", "Diabetes_binary").show(truncate=False)
df1=output.select("features", "Diabetes_binary")
train_df1, test_df1 = df1.randomSplit([0.8, 0.2])

print ("num of train",train_df1.cache().count())
print ("num of test",test_df1.cache().count())

22/10/10 12:23:36 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------------------------------------------------------------------------------------------+---------------+
|features                                                                                        |Diabetes_binary|
+------------------------------------------------------------------------------------------------+---------------+
|[1.0,1.0,1.0,6.164414002968976,0.0,0.0,0.0,0.0,1.0,1.0,0.0,4.0,0.0,6.0,1.0,0.0,12.0,1.0,4.0]    |0              |
|[0.0,1.0,1.0,4.898979485566356,1.0,1.0,0.0,1.0,1.0,0.0,0.0,5.0,30.0,30.0,1.0,0.0,7.0,1.0,3.0]   |0              |
|[1.0,1.0,1.0,5.291502622129181,1.0,0.0,1.0,0.0,0.0,1.0,0.0,3.0,30.0,30.0,1.0,0.0,8.0,1.0,1.0]   |0              |
|(19,[0,2,3,4,7,11,12,13,16,17,18],[1.0,1.0,5.196152422706632,1.0,1.0,5.0,30.0,25.0,8.0,1.0,1.0])|0              |
|(19,[2,3,4,7,8,9,11,15,16,17,18],[1.0,5.385164807134504,1.0,1.0,1.0,1.0,3.0,1.0,12.0,1.0,6.0])  |0              |
|[0.0,1.0,1.0,4.47213595499958,0.0,0.0,1.0,1.0,1.0,1.0,0.0,5.0,28.0,20.0,0.0,0.0

num of train 49516


num of test 21171


In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString,StringIndexer,VectorIndexer
from pyspark.ml.classification import DecisionTreeClassificationModel,DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

labelIndexer = StringIndexer().setInputCol("Diabetes_binary").setOutputCol("indexedLabel").fit(df1)
featureIndexer = VectorIndexer().setInputCol("features").setOutputCol("indexedFeatures").setMaxCategories(19).fit(df1)
labelConverter = IndexToString().setInputCol("prediction").setOutputCol("predictedLabel").setLabels(labelIndexer.labels)

dtClassifier = DecisionTreeClassifier(impurity='gini', maxDepth=5, maxBins=32).setLabelCol("indexedLabel").setFeaturesCol("indexedFeatures")
pipelinedClassifier = Pipeline().setStages([labelIndexer, featureIndexer, dtClassifier, labelConverter])
modelClassifier = pipelinedClassifier.fit(train_df1)
predictionsClassifier = modelClassifier.transform(test_df1)
predictionsClassifier.select("predictedLabel", "Diabetes_binary", "features").show(20)

+--------------+---------------+--------------------+
|predictedLabel|Diabetes_binary|            features|
+--------------+---------------+--------------------+
|             1|              1|(19,[0,1,2,3,4,5,...|
|             1|              1|(19,[0,1,2,3,4,5,...|
|             1|              1|(19,[0,1,2,3,4,5,...|
|             1|              1|(19,[0,1,2,3,4,5,...|
|             1|              0|(19,[0,1,2,3,4,5,...|
|             1|              0|(19,[0,1,2,3,4,5,...|
|             1|              0|(19,[0,1,2,3,4,5,...|
|             1|              0|(19,[0,1,2,3,4,5,...|
|             1|              1|(19,[0,1,2,3,4,6,...|
|             1|              1|(19,[0,1,2,3,4,6,...|
|             1|              1|(19,[0,1,2,3,4,6,...|
|             1|              1|(19,[0,1,2,3,4,6,...|
|             1|              0|(19,[0,1,2,3,4,6,...|
|             1|              1|(19,[0,1,2,3,4,6,...|
|             1|              1|(19,[0,1,2,3,4,6,...|
|             1|            

In [4]:
evaluatorClassifier = MulticlassClassificationEvaluator().setLabelCol("indexedLabel").setPredictionCol("prediction").setMetricName("accuracy")
 
accuracy = evaluatorClassifier.evaluate(predictionsClassifier)
 
print("Accuracy = " + str(accuracy))

Accuracy = 0.7186245335600586


In [11]:
treeModelClassifier = modelClassifier.stages[2]
 
print("Learned classification tree model:\n" + str(treeModelClassifier.toDebugString))

Learned classification tree model:
DecisionTreeClassificationModel: uid=DecisionTreeClassifier_0ddbd0cdfdea, depth=5, numNodes=35, numClasses=2, numFeatures=19
  If (feature 0 in {1.0})
   If (feature 3 <= 5.522494968940841)
    If (feature 11 in {0.0,2.0,3.0,4.0})
     If (feature 1 in {0.0})
      If (feature 11 in {3.0,4.0})
       Predict: 0.0
      Else (feature 11 not in {3.0,4.0})
       Predict: 1.0
     Else (feature 1 not in {0.0})
      Predict: 0.0
    Else (feature 11 not in {0.0,2.0,3.0,4.0})
     If (feature 16 in {0.0,1.0,2.0,3.0,4.0,5.0,6.0})
      Predict: 1.0
     Else (feature 16 not in {0.0,1.0,2.0,3.0,4.0,5.0,6.0})
      If (feature 18 in {0.0,1.0,2.0,3.0})
       Predict: 0.0
      Else (feature 18 not in {0.0,1.0,2.0,3.0})
       Predict: 1.0
   Else (feature 3 > 5.522494968940841)
    Predict: 0.0
  Else (feature 0 not in {1.0})
   If (feature 11 in {2.0,3.0,4.0})
    If (feature 1 in {1.0})
     If (feature 3 <= 5.243827522417907)
      If (feature 11 in {3.0,

ModuleNotFoundError: No module named 'sklearn'